In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime

import MP_functions as dfutil

# Importing & Loading Data

Make sure to unzip the pickles.zip into the root folder for the project.

In [9]:
# from load_pickles import df as pickle
df = dfutil.open_dataframe_pickle('MTA_DATA_SPRING_2014_to_2016_FULL.pickle')
dtp = dfutil.open_dataframe_pickle('daily_throughput.pickle')
htp = dfutil.open_dataframe_pickle('hourly_throughput.pickle')

# Functions

In [10]:
def get_daily_counts(row, max_counter, cols):
    """
    cols - a list of columns to compute the difference from with format [x, y] = x-y
    """
    counter = row[cols[0]] - row[cols[1]]
    if counter < 0:
        # May be counter is reversed?
        counter = -counter
    if counter > max_counter:
#         print(row[cols[0]], row[cols[1]])
        counter = min(row[cols[0]], row[cols[1]])
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

In [11]:
def cleanData(frame):
    # Strip whitespace from column names
    frame.columns = [s.strip() for s in frame.columns.values]

    # Create datetime column with datetime datatype
    frame['Datetime'] = pd.to_datetime(frame.DATE + ' ' + frame.TIME,
                                        format = '%m/%d/%Y %H:%M:%S')
    frame['Day_of_week'] = frame['Datetime'].apply(lambda row: row.strftime("%A"))
    
    # Chain methods together to further clean data:
        # drop old date and time columns
        # rename columns
    
    dict_col_rename = {'C/A' : 'C_A', 'UNIT' : 'Unit', 'STATION' : 'Station', 'LINENAME' : 'Linename',
                      'DIVISION' : 'Division', 'DESC' : 'Desc', 'ENTRIES' : 'Entries', 'EXITS' : 'Exits',
                      'DATE' : 'Date'}    

    frame2 = frame.rename(columns = dict_col_rename)
    
    # Check uniqueness of rows/indexes by getting counts.
    (frame2
     .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])
     .count() #.Entries.count()
     .reset_index()
     .sort_values('Entries', ascending = False)
    )
    
    # Drop duplicates.
    return frame2.drop_duplicates(subset=['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])
    
    

In [12]:
def calcDailyThroughput(frame):
    
    # group daily entries and daily exits
    daily_entries = (frame
                      .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
                      .Entries
                      .first()
                      .reset_index()
                    )

    daily_exits = (frame
                    .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
                    .Exits
                    .first()
                    .reset_index()
                   )
    
    # Calculate the differences by day
    daily_entries[["Prev_date", "Prev_entries"]] = (daily_entries
                                                     .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Entries"]
                                                     .transform(lambda grp: grp.shift(1)))

    daily_exits[["Prev_date", "Prev_exits"]]   = (daily_exits
                                                   .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Exits"]
                                                   .transform(lambda grp: grp.shift(1)))

    # Drop all the null values generated above
    daily_entries.dropna(subset=["Prev_date"], axis=0, inplace=True)
    daily_exits.dropna(subset=["Prev_date"], axis=0, inplace=True)


    daily_entries["Daily_Entries"] = daily_entries.apply(get_daily_counts, axis=1, args=(1000000, ['Entries', 'Prev_entries']))
    daily_exits["Daily_Exits"] = daily_exits.apply(get_daily_counts, axis=1, args=(1000000, ['Exits', 'Prev_exits']))
    
    daily = pd.merge(daily_entries, daily_exits, on=['C_A','Unit','SCP', 'Station', 'Date', 'Prev_date'])
    daily['Total_throughput'] = daily['Daily_Entries'] + daily['Daily_Exits']
    
    return (daily
             .groupby(['Station', 'Date'])
             .sum()
            #  .sort_values(by=['Total_throughput'], ascending=False)
             .loc[:,['Daily_Entries', 'Daily_Exits', 'Total_throughput']]
            #  .frame()
            )

In [13]:
def calcHourThroughput(frame):
    shift = frame.copy()

    shift[['Datetime_Prev', 'Entries_Prev', 'Exits_Prev']] = (shift
                .groupby(['C_A', 'Unit', 'SCP', 'Station'])['Datetime', 'Entries', 'Exits']
                .transform(lambda grp: grp.shift(1)))
    
    shift['Entries'] = shift['Entries'] - shift['Entries_Prev']
    shift['Exits'] = shift['Exits'] - shift['Exits_Prev']
    shift = shift.dropna(how = 'any')
    
    shift['Throughput'] = shift['Entries'] + shift['Exits']
    
    return shift.loc[:,['Station','Datetime','Throughput']]

In [14]:
# Make note to save and open pickles here:
# dfutil.save_dataframe_as_pickle(df,'MTA_DATA_SPRING_2014_to_2016_FULL.pickle')
# df_pickle = dfutil.open_dataframe_pickle('MTA_DATA_SPRING_2014_to_2016_FULL.pickle') # returns a df

# Function Testing

In [15]:
# To find hour through put 
# test_df3.groupby(['Station', 'Datetime']).sum()

In [16]:
df.head()

,C_A,Unit,SCP,Station,Linename,Division,Date,TIME,Desc,Entries,Exits,Datetime,Day_of_week
0,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,00:00:00,REGULAR,5704958,1934814,2016-06-18 00:00:00,Saturday
1,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,04:00:00,REGULAR,5704958,1934829,2016-06-18 04:00:00,Saturday
2,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,08:00:00,REGULAR,5704958,1934886,2016-06-18 08:00:00,Saturday
3,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,12:00:00,REGULAR,5704958,1934993,2016-06-18 12:00:00,Saturday
4,A002,R051,02-00-00,59 ST,NQR456,BMT,06/18/2016,16:00:00,REGULAR,5704958,1935069,2016-06-18 16:00:00,Saturday


In [18]:
htp.head()

,Station,Datetime,Throughput
1,59 ST,2016-06-18 04:00:00,15.0
2,59 ST,2016-06-18 08:00:00,57.0
3,59 ST,2016-06-18 12:00:00,107.0
4,59 ST,2016-06-18 16:00:00,76.0
5,59 ST,2016-06-18 20:00:00,63.0


In [19]:
htp.groupby(['Station', 'Datetime']).sum()

Throughput
Station    Datetime                       
1 AVE      2015-02-28 03:00:00   -642981.0
           2015-02-28 07:00:00      1008.0
           2015-02-28 11:00:00      4564.0
           2015-02-28 15:00:00      8289.0
           2015-02-28 19:00:00     10733.0
           2015-02-28 23:00:00     10352.0
           2015-03-01 03:00:00      4968.0
           2015-03-01 07:00:00      1010.0
           2015-03-01 11:00:00      3297.0
           2015-03-01 15:00:00      7884.0
           2015-03-01 19:00:00      8695.0
           2015-03-01 23:00:00      5538.0
           2015-03-02 03:00:00      1412.0
           2015-03-02 07:00:00      1930.0
           2015-03-02 11:00:00     13928.0
           2015-03-02 15:00:00      7776.0
           2015-03-02 19:00:00     14161.0
           2015-03-02 23:00:00      9171.0
           2015-03-03 03:00:00      1603.0
           2015-03-03 07:00:00      2045.0
           2015-03-03 11:00:00     14351.0
           2015-03-03 15:00:00      7893.0
           2015-03-03 19:00:00     15905.0
           2015-03-03 23:00:00     10250.0
           2015-03-04 03:00:00      1665.0
           2015-03-04 07:00:00      1950.0
           2015-03-04 11:00:00     14268.0
           2015-03-04 15:00:00      7566.0
           2015-03-04 19:00:00     14502.0
           2015-03-04 23:00:00      9907.0
...                                    ...
ZEREGA AVE 2016-06-20 01:00:00       272.0
           2016-06-20 05:00:00       133.0
           2016-06-20 09:00:00      1462.0
           2016-06-20 13:00:00       804.0
           2016-06-20 17:00:00      1093.0
           2016-06-20 21:00:00      1092.0
           2016-06-21 01:00:00       404.0
           2016-06-21 05:00:00       129.0
           2016-06-21 09:00:00      1504.0
           2016-06-21 13:00:00       786.0
           2016-06-21 17:00:00      1096.0
           2016-06-21 21:00:00      1170.0
           2016-06-22 01:00:00       365.0
           2016-06-22 05:00:00       142.0
           2016-06-22 09:00:00      1477.0
           2016-06-22 13:00:00       794.0
           2016-06-22 17:00:00      1074.0
           2016-06-22 21:00:00      1174.0
           2016-06-23 01:00:00       425.0
           2016-06-23 05:00:00       138.0
           2016-06-23 09:00:00      1515.0
           2016-06-23 13:00:00       737.0
           2016-06-23 17:00:00      1046.0
           2016-06-23 21:00:00      1114.0
           2016-06-24 01:00:00       349.0
           2016-06-24 05:00:00       134.0
           2016-06-24 09:00:00      1478.0
           2016-06-24 13:00:00       815.0
           2016-06-24 17:00:00      1108.0
           2016-06-24 21:00:00      1113.0

[956098 rows x 1 columns]

In [103]:
# dtp['Day_of_week'] = #(pd.to_datetime(dtp.reset_index()['Date'], format = '%m/%d/%Y'))

In [137]:
dtp = dtp.reset_index()
dtp['Day_of_week'] = pd.to_datetime(dtp['Date'], format = '%m/%d/%Y').apply(lambda row: row.strftime("%A"))

dtp_sum = dtp.groupby(['Day_of_week','Station'])['Total_throughput'].sum().reset_index()
pivot = dtp_sum.pivot(index='Station', columns='Day_of_week', values = 'Total_throughput')

monday = pivot['Monday'].sort_values(ascending=False).index.values
tuesday = pivot['Tuesday'].sort_values(ascending=False).index.values
wednesday = pivot['Wednesday'].sort_values(ascending=False).index.values
thursday = pivot['Thursday'].sort_values(ascending=False).index.values
friday = pivot['Friday'].sort_values(ascending=False).index.values
saturday = pivot['Saturday'].sort_values(ascending=False).index.values
sunday = pivot['Sunday'].sort_values(ascending=False).index.values

df = pd.DataFrame({'Monday': monday,
                  'Tuesday': tuesday,
                  'Wednesday': wednesday,
                  'Thursday': thursday,
                  'Friday': friday,
                  'Saturday': saturday,
                  'Sunday': sunday})

In [140]:
dfutil.save_dataframe_as_pickle(df,'ranked_stations_by_day.pickle')

# TODO

Day of the week - 7 days 
- index'd by rank
- names of stations 
- each column is a time of the day

map of morning, afternoon, evening
     for each day of week


In [86]:
dtp

Daily_Entries  Daily_Exits  Total_throughput
Station    Date                                                    
1 AVE      02/28/2015      2149122.0     341314.0         2490436.0
           02/28/2016      2153712.0     342328.0         2496040.0
           02/29/2016        14018.0      15107.0           29125.0
           03/01/2015      2151936.0     342214.0         2494150.0
           03/01/2016      2156668.0     343143.0         2499811.0
           03/02/2015      2154140.0     342541.0         2496681.0
           03/02/2016      2159107.0     343636.0         2502743.0
           03/03/2015      2154068.0     342652.0         2496720.0
           03/03/2016      2158936.0     343655.0         2502591.0
           03/04/2015      2150710.0     342596.0         2493306.0
           03/04/2016      2155632.0     343580.0         2499212.0
           03/05/2015      2150726.0     342514.0         2493240.0
           03/05/2016      2156535.0     343656.0         2500191.0
           03/06/2016        20011.0      21791.0           41802.0
           03/07/2015      2146870.0     342550.0         2489420.0
           03/07/2016      2149822.0     343268.0         2493090.0
           03/08/2015      2145564.0     342297.0         2487861.0
           03/08/2016      2150316.0     343194.0         2493510.0
           03/09/2015      2147206.0     342648.0         2489854.0
           03/09/2016      2151783.0     343643.0         2495426.0
           03/10/2015      2146387.0     342610.0         2488997.0
           03/10/2016      2151705.0     343552.0         2495257.0
           03/11/2015      2145743.0     342412.0         2488155.0
           03/11/2016      2152120.0     343393.0         2495513.0
           03/12/2015      2145805.0     342452.0         2488257.0
           03/12/2016      2151444.0     343677.0         2495121.0
           03/13/2015      2144862.0     342644.0         2487506.0
           03/13/2016      2149273.0     343645.0         2492918.0
           03/14/2015      2137977.0     342647.0         2480624.0
           03/14/2016      2140598.0     343245.0         2483843.0
...                              ...          ...               ...
ZEREGA AVE 06/11/2015       756273.0     705710.0         1461983.0
           06/11/2016       759119.0     708101.0         1467220.0
           06/12/2015       756613.0     705862.0         1462475.0
           06/12/2016       758034.0     707214.0         1465248.0
           06/13/2015       755509.0     704946.0         1460455.0
           06/13/2016       756632.0     706040.0         1462672.0
           06/14/2015       755343.0     704754.0         1460097.0
           06/14/2016       758022.0     707192.0         1465214.0
           06/15/2015       757055.0     706078.0         1463133.0
           06/15/2016       759716.0     708502.0         1468218.0
           06/16/2015       757361.0     706304.0         1463665.0
           06/16/2016       760171.0     708844.0         1469015.0
           06/17/2015       757753.0     706566.0         1464319.0
           06/17/2016       760479.0     709016.0         1469495.0
           06/18/2015       758014.0     706700.0         1464714.0
           06/18/2016       760688.0     709127.0         1469815.0
           06/19/2015       758242.0     706811.0         1465053.0
           06/19/2016       759717.0     708156.0         1467873.0
           06/20/2015       757393.0     706015.0         1463408.0
           06/20/2016       758461.0     707110.0         1465571.0
           06/21/2015       757334.0     706385.0         1463719.0
           06/21/2016       759965.0     708742.0         1468707.0
           06/22/2015       759091.0     708071.0         1467162.0
           06/22/2016       761784.0     710428.0         1472212.0
           06/23/2015       759371.0     708280.0         1467651.0
           06/23/2016       762036.0     710701.0         1472737.0
           

# Daily

In [85]:
dtp['Day_of_week'] = dtp['Datetime'].apply(lambda row: row.strftime("%A"))

KeyError: 'Datetime'

In [22]:
htp['Day_of_week'] = htp['Datetime'].apply(lambda row: row.strftime("%A"))

In [41]:
htp['Hour'] = htp['Datetime'].apply(lambda row: row.strftime("%-I%p"))

In [63]:
eight_am_mask = (htp['Hour'] == '8AM')
four_pm_mask = (htp['Hour'] == '4PM')
twelve_pm_mask = (htp['Hour'] == '12PM')

In [69]:
htp_8am = htp.loc[eight_am_mask,:].groupby(['Station','Day_of_week','Hour']).sum().reset_index()

In [77]:
htp.groupby(['Hour']).count()

,Station,Datetime,Throughput,Day_of_week
Hour,,,,
10AM,52570,52570,52570,52570
10PM,45428,45428,45428,45428
11AM,83247,83247,83247,83247
11PM,80074,80074,80074,80074
12AM,571223,571223,571223,571223
12PM,579242,579242,579242,579242
1AM,374861,374861,374861,374861
1PM,381062,381062,381062,381062
2AM,43981,43981,43981,43981


In [75]:
htp_8am.groupby(['Day_of_week']).max()

,Station,Hour,Throughput
Day_of_week,,,
Friday,ZEREGA AVE,8AM,8.384000e+07
Monday,ZEREGA AVE,8AM,7.694750e+05
Saturday,ZEREGA AVE,8AM,2.666094e+06
Sunday,ZEREGA AVE,8AM,1.149430e+05
Thursday,ZEREGA AVE,8AM,6.201800e+05
Tuesday,ZEREGA AVE,8AM,8.487320e+05
Wednesday,ZEREGA AVE,8AM,3.182406e+09


# Data Exploration 
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

## Data Cleaning

In [ ]:
# The column names contain unneeded whitespace.
df1.columns.values

In [ ]:
# The dataframe also lacks a timeseries.
df1.dtypes

In [ ]:
# Strip whitespace from column names
df1.columns = [s.strip() for s in df1.columns.values]

# Create datetime column with datetime datatype
df1['Datetime'] = pd.to_datetime(df1.DATE + ' ' + df1.TIME,
                                 format = '%m/%d/%Y %H:%M:%S')

df1.iloc[:5]

In [ ]:
# Chain methods together to further clean data:
    # drop old date and time columns
    # rename columns
    
dict_col_rename = {'C/A' : 'C_A', 'UNIT' : 'Unit', 'STATION' : 'Station', 'LINENAME' : 'Linename',
                  'DIVISION' : 'Division', 'DESC' : 'Desc', 'ENTRIES' : 'Entries', 'EXITS' : 'Exits',
                  'DATE' : 'Date'}    

df2 = (df1
       #.drop('DATE', axis = 1)
       .drop('TIME', axis = 1)
       .rename(columns = dict_col_rename)
      )

df2.iloc[:5]

# Removing Erroneous Observations

In [ ]:
# Check uniqueness of rows/indexes by getting counts.
(df2
 .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])
 .count() #.Entries.count()
 .reset_index()
 .sort_values('Entries', ascending = False)
 .iloc[:5]
)

In [ ]:
# On 3/25, we don't seem to have two entries for same time, but let's take a look anyway.

mask = ((df2["C_A"] == "A002") & 
(df2["Unit"] == "R051") & 
(df2["SCP"] == "02-00-00") & 
(df2["Station"] == "59 ST") &
(df2["Datetime"].dt.date == datetime.datetime(2017, 3, 25).date()))
df2[mask].head()

In [ ]:
df2.Desc.value_counts()

In [ ]:
# Questions for the future, if there is time:
    # Are there other values of DESC?
    # Are there other fields to check for odd values?

# Drop duplicates.
df_no_dupe = df2.drop_duplicates(subset=['C_A', 'Unit', 'SCP', 'Station', 'Datetime'])

# Check uniqueness again after data cleaning to confirm cleanness.

In [ ]:
df_no_dupe.head()

# Calculating the Entries and Exits per Day

In [ ]:
df_daily_entries = (df_no_dupe
            .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
            .Entries
            .first()
            .reset_index()
           )

df_daily_exits = (df_no_dupe
            .groupby(['C_A', 'Unit', 'SCP', 'Station', 'Date'])
            .Exits
            .first()
            .reset_index()
           )


In [ ]:
# Calculate the differences by day
df_daily_entries[["Prev_date", "Prev_entries"]] = (df_daily_entries
                                                       .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Entries"]
                                                       .transform(lambda grp: grp.shift(1)))

df_daily_exits[["Prev_date", "Prev_exits"]]   = (df_daily_exits
                                                       .groupby(["C_A", "Unit", "SCP", "Station"])["Date", "Exits"]
                                                       .transform(lambda grp: grp.shift(1)))

# Drop all the null values generated above
df_daily_entries.dropna(subset=["Prev_date"], axis=0, inplace=True)
df_daily_exits.dropna(subset=["Prev_date"], axis=0, inplace=True)

In [ ]:
df_daily_entries.head()

In [ ]:
# Check for any counters that have been reversed
df_daily_entries[df_daily_entries["Entries"] < df_daily_entries["Prev_entries"]].head()

# WTC: Is this something that can be solved by sorting before applying the transform above?

In [ ]:
##### HAVE NOT EDITED YET

# Pick a value from one of the counters reversed above & check for it 
# What's the deal with counter being in reverse
# mask = ((turnstiles_df["C/A"] == "A011") & 
# (turnstiles_df["UNIT"] == "R080") & 
# (turnstiles_df["SCP"] == "01-00-00") & 
# (turnstiles_df["STATION"] == "57 ST-7 AV") &
# (turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2016, 8, 27).date()))
# turnstiles_df[mask].head()

In [ ]:
# Let's see how many stations have this problem

(df_daily_entries[df_daily_entries["Entries"] < df_daily_entries["Prev_entries"]]
    .groupby(["C_A", "Unit", "SCP", "Station"])
    .size())

In [ ]:
# def get_daily_counts(row, max_counter, cols):
#     counter = row[cols[0]] - row[cols[1]]
#     if counter < 0:
#         # May be counter is reversed?
#         counter = -counter
#     if counter > max_counter:
#         print(row[cols[0]], row[cols[1]])
#         counter = min(row[cols[0]], row[cols[1]])
#     if counter > max_counter:
#         # Check it again to make sure we are not giving a counter that's too big
#         return 0
#     return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
df_daily_entries["Daily_Entries"] = df_daily_entries.apply(get_daily_counts, axis=1, args=(1000000, ['Entries', 'Prev_entries']))
df_daily_exits["Daily_Exits"] = df_daily_exits.apply(get_daily_counts, axis=1, args=(1000000, ['Exits', 'Prev_exits']))

In [ ]:
df_daily_entries.head()

In [ ]:
df_daily_exits.head()

In [ ]:
df_daily = pd.merge(df_daily_entries, df_daily_exits, on=['C_A','Unit','SCP', 'Station', 'Date', 'Prev_date'])
df_daily['Total_throughput'] = df_daily['Daily_Entries'] + df_daily['Daily_Exits']
df_daily.head()

In [ ]:
(df_daily
 .groupby(['Station', 'Date'])
 .sum()
#  .sort_values(by=['Total_throughput'], ascending=False)
 .loc[:,['Daily_Entries', 'Daily_Exits', 'Total_throughput']]
#  .frame()
)

# Calculating the Entries and Exits per Hour


In [ ]:
# Use concat (as a join) to fix ("de-cumulate") the columns Entries and Exits

df_shift = (df_no_dupe
            .copy()
            .drop('Linename', axis = 1)
            .drop('Division', axis = 1)
           )

df_shift[['Datetime_Prev', 'Entries_Prev', 'Exits_Prev']] = (df_shift
            .groupby(['C_A', 'Unit', 'SCP', 'Station'])['Datetime', 'Entries', 'Exits']
            .transform(lambda grp: grp.shift(1)))

df_shift.head()


# Legacy

# df_shift.columns
# df_shift['Datetime_Prev', 'Entries_Prev', 'Exits_Prev'] = (
#     df_no_dupe#[['C_A', 'Unit', 'SCP', 'Station', 'Datetime', 'Entries', 'Exits']]
#             .groupby(['C_A', 'Unit', 'SCP', 'Station'])['Datetime', 'Entries', 'Exits']
#             #.groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
#             .transform(lambda grp: grp.shift(1))
#             #.shift(periods = 1)
#             #.rename(columns = {'Entries' : 'Entries_Shift', 'Exits' : 'Exits_Shift', 
#             #                   'Datetime' : 'Prev_datetime'})
#            )

In [ ]:
df_shift['Entries'] = df_shift['Entries'] - df_shift['Entries_Prev']
df_shift['Exits'] = df_shift['Exits'] - df_shift['Exits_Prev']
df_shift = df_shift.dropna(how = 'any')

df_shift.head()

In [ ]:
# Challenge 3 - Total Daily Entries
#df3or4['Datetime'].dt.date == datetime.datetime(YYYY, MM, DD).date()

In [ ]:
# def get_daily_counts(row, max_counter):
#     counter = abs(row["ENTRIES"] - row["PREV_ENTRIES"])
    
#     if counter > max_counter:
#         print(row["ENTRIES"], row["PREV_ENTRIES"])
#         return 0
#     return counter

# # If counter is > 1Million, then the counter might have been reset.  
# # Just set it to zero as different counters have different cycle limits
# _ = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)